In [ ]:
import numpy as np 
import pandas as pd

mkt_data = pd.read_csv('synthetic_ecommerce_data.csv') #importing the file


In [7]:
mkt_data.head(5) #viewing the first few rows

,Transaction_ID,Customer_ID,Product_ID,Transaction_Date,Units_Sold,Discount_Applied,Revenue,Clicks,Impressions,Conversion_Rate,Category,Region,Ad_CTR,Ad_CPC,Ad_Spend
0,8b460852-7c64-46fa-998b-b0976879d082,Customer_65,Product_224,2024-10-06,134,0.14,305.54,11,65,0.17,Electronics,Europe,0.0180,0.55,9.90
1,418612e7-8744-4ba3-bb0c-105b47e2a968,Customer_1910,Product_584,2024-10-29,109,0.30,1102.19,15,201,0.07,Home Appliances,Asia,0.1589,0.40,63.56
2,5bc3b98f-cb0c-4b12-947c-df8bbb35a73e,Customer_2306,Product_374,2024-04-04,116,0.04,471.29,16,199,0.08,Toys,Asia,0.0596,1.50,89.40
3,28fb67c8-e8c0-447a-841c-f760730de0eb,Customer_17206,Product_220,2024-08-25,125,0.20,980.26,12,355,0.03,Clothing,Europe,0.0444,0.44,19.54
4,8bee087a-a8a9-45bb-89d7-04d1710f1b00,Customer_16033,Product_358,2024-05-05,132,0.07,803.76,44,355,0.12,Books,North America,0.1270,0.53,67.31


In [6]:
mkt_data.isnull().sum() #viewing if there is a null data

Transaction_ID      0
Customer_ID         0
Product_ID          0
Transaction_Date    0
Units_Sold          0
Discount_Applied    0
Revenue             0
Clicks              0
Impressions         0
Conversion_Rate     0
Category            0
Region              0
Ad_CTR              0
Ad_CPC              0
Ad_Spend            0
dtype: int64

In [9]:
mkt_data.columns = mkt_data.columns.str.lower() #fixing the name to lowercase
mkt_data.head(5)

,transaction_id,customer_id,product_id,transaction_date,units_sold,discount_applied,revenue,clicks,impressions,conversion_rate,category,region,ad_ctr,ad_cpc,ad_spend
0,8b460852-7c64-46fa-998b-b0976879d082,Customer_65,Product_224,2024-10-06,134,0.14,305.54,11,65,0.17,Electronics,Europe,0.0180,0.55,9.90
1,418612e7-8744-4ba3-bb0c-105b47e2a968,Customer_1910,Product_584,2024-10-29,109,0.30,1102.19,15,201,0.07,Home Appliances,Asia,0.1589,0.40,63.56
2,5bc3b98f-cb0c-4b12-947c-df8bbb35a73e,Customer_2306,Product_374,2024-04-04,116,0.04,471.29,16,199,0.08,Toys,Asia,0.0596,1.50,89.40
3,28fb67c8-e8c0-447a-841c-f760730de0eb,Customer_17206,Product_220,2024-08-25,125,0.20,980.26,12,355,0.03,Clothing,Europe,0.0444,0.44,19.54
4,8bee087a-a8a9-45bb-89d7-04d1710f1b00,Customer_16033,Product_358,2024-05-05,132,0.07,803.76,44,355,0.12,Books,North America,0.1270,0.53,67.31


1. Customer Lifetime Value (CLV) Analysis

In [51]:
# Convert transaction_date to datetime
mkt_data['transaction_date'] = pd.to_datetime(mkt_data['transaction_date'])

# Extract the year from the datetime object
mkt_data['year'] = mkt_data['transaction_date'].dt.year

# grouping by year
sum_revenue= mkt_data.groupby(['year'])['revenue'].sum()

#To get the header
#print(sum_revenue.to_string(index=True, header=True)) 

#groupbyregion 
grp_year= mkt_data.groupby(['year','region'])['revenue'].sum()
grp_year

year  region       
2023  Asia              1229401.17
      Europe            1252051.13
      North America     1241617.47
2024  Asia             17263469.96
      Europe           17146930.20
      North America    17177611.14
Name: revenue, dtype: float64

2. Sales Performance by Product

In [54]:
# Total revenue per customer
clv = mkt_data.groupby('customer_id')['revenue'].sum().reset_index()
clv.rename(columns={'revenue': 'total_revenue'}, inplace=True)

# Average revenue per transaction
avg_transaction_value = mkt_data.groupby('customer_id')['revenue'].mean().reset_index()
avg_transaction_value.rename(columns={'revenue': 'avg_transaction_value'}, inplace=True)

# Merging for CLV analysis
clv = clv.merge(avg_transaction_value, on='customer_id')
print(clv.head())

      customer_id  total_revenue  avg_transaction_value
0      Customer_1        2932.58             586.516000
1     Customer_10        2925.10             975.033333
2    Customer_100        2927.21             585.442000
3   Customer_1000        2208.97             736.323333
4  Customer_10000        2650.34             883.446667


In [57]:
# Group by product to calculate total revenue and units sold
product_performance = mkt_data.groupby('product_id').agg({
    'units_sold': 'sum',
    'revenue': 'sum',
    'discount_applied': 'mean'
}).reset_index()

# Sort by revenue
product_performance = product_performance.sort_values(by='revenue', ascending=False)
print(product_performance.head(10))

      product_id  units_sold    revenue  discount_applied
11   Product_108       15849  125682.19          0.138803
340  Product_404       13930  125115.04          0.140420
302  Product_370       15338  123196.60          0.130513
652  Product_686       13338  121787.93          0.125752
745   Product_77       13610  121302.64          0.147857
348  Product_411       16323  121283.88          0.147949
177  Product_258       15216  121225.16          0.141102
998  Product_998       15822  120448.50          0.150000
657  Product_690       14063  120414.76          0.130893
738  Product_763       13125  120157.60          0.158649


In [60]:
# Calculate ROAS (Return on Ad Spend)
mkt_data['roas'] = mkt_data['revenue'] / mkt_data['ad_spend']

# Average ROAS by region
roas_by_region = mkt_data.groupby('region')['roas'].mean().reset_index()
print(roas_by_region)

          region       roas
0           Asia  13.751605
1         Europe  13.574929
2  North America  13.656352


In [61]:
# Extract month and analyze trends
mkt_data['month'] = mkt_data['transaction_date'].dt.month

# Revenue trends by month
monthly_trends = mkt_data.groupby('month')['revenue'].sum().reset_index()
print(monthly_trends)

   month      revenue
0      1  55311081.07
